In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline 

# What is the causal impact of having a PhD on making over \$50K?

As of v0.27.x, ktrain supports causal inference using [meta-learners](https://arxiv.org/abs/1706.03461). We will use the well-stuided [Adults](https://raw.githubusercontent.com/amaiya/ktrain/master/ktrain/tests/tabular_data/adults.csv) dataset from the UCI ML repository, which is census data from the early to mid 1990s.  The objective is estimate how much earning a PhD increases the probability of making over $50K.  This dataset is simply being used as a simple demonstration example.  Unlike conventional supervised machine learning, there is typically no ground truth for causal infernence models, unless you're using a simulated datasets.  So, we will simply check our estimates to see if they agree with intuition for illustration purposes.

Let's begin by loading the dataset and creating a binary treatment (1 for PhD and 0 for no PhD).

In [19]:
!wget https://raw.githubusercontent.com/amaiya/ktrain/master/ktrain/tests/tabular_data/adults.csv -O /tmp/adults.csv

--2021-07-15 23:15:06--  https://raw.githubusercontent.com/amaiya/ktrain/master/ktrain/tests/tabular_data/adults.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4573758 (4.4M) [text/plain]
Saving to: ‘/tmp/adults.csv’

/tmp/adults.csv     100%[===================>]   4.36M  --.-KB/s    in 0.06s   

2021-07-15 23:15:06 (75.5 MB/s) - ‘/tmp/adults.csv’ saved [4573758/4573758]



In [20]:
import pandas as pd
df = pd.read_csv('/tmp/adults.csv')
df = df.rename(columns=lambda x: x.strip())
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x) 
filter_set = 'Doctorate'
df['treatment'] = df['education'].apply(lambda x: 1 if x in filter_set else 0)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,treatment
0,25,Private,178478,Bachelors,13,Never-married,Tech-support,Own-child,White,Female,0,0,40,United-States,<=50K,0
1,23,State-gov,61743,5th-6th,3,Never-married,Transport-moving,Not-in-family,White,Male,0,0,35,United-States,<=50K,0
2,46,Private,376789,HS-grad,9,Never-married,Other-service,Not-in-family,White,Male,0,0,15,United-States,<=50K,0
3,55,?,200235,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,50,United-States,>50K,0
4,36,Private,224541,7th-8th,4,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,El-Salvador,<=50K,0


Next, let's invoke the `causal_inference_model` function to create a `CausalInferenceModel` instance and invoke `fit` to estimate the individualized treatment effect for each row in this dataset.  By default, a [T-Learner](https://arxiv.org/abs/1706.03461) metalearner is used with LightGBM models as base learners. Note that we are ignoring the education-related columns as they are already captured in the treatment in addition to the`fnlwget` column.

In [21]:
from ktrain.tabular import causal_inference_model
cm = causal_inference_model(df,
                            treatment_col='treatment', 
                            outcome_col='class',
                            ignore_cols=['fnlwgt', 'education','education-num']).fit()

replaced ['<=50K', '>50K'] in column "class" with [0, 1]
outcome column (categorical): class
treatment column: treatment
numerical/categorical covariates: ['age', 'workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
preprocess time:  0.4675877094268799  sec
start fitting causal inference model
time to fit causal inference model:  0.8784487247467041  sec


The overall average treatment effect for all examples is 0.20.  That is, having a PhD increases your probability of making over $50K by 20 percentage points.

In [22]:
cm.estimate_ate()

{'ate': 0.20340645077516034}

For those with Master's degrees, we find that it is lower (as expected):

In [23]:
cm.estimate_ate(cm.df['education'] == 'Masters')

{'ate': 0.17672418257642838}

For those that dropped out of school, we find that it is higher (also as expected):

In [24]:
cm.estimate_ate(cm.df['education'].isin(['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th', '12th']))

{'ate': 0.2586697863578173}

As shown above, we can examine how the treatment effect varies across units in the population.  This is referred as [uplift modeling](https://en.wikipedia.org/wiki/Uplift_modelling) and is often used by companies for targetd promotions by identifying those individuals with the highest treatment effects.

**ktrain** uses the **CausalNLP** package for inferring causality.  For more information, see the [CausalNLP documentation](https://amaiya.github.io/causalnlp).